# <center>Майнор "Интеллектуальный анализ данных"</center>

# <center>Курс "Введение в анализ данных"</center>

# <center>Лабораторная работа №3. Supervised Learning</center>

## Данные

В рамках данной лабораторной работы вам предлагается проанализировать набор данных о студентах двух школ в Португалии.  
В файле `students_data.csv` представлена информация о студентах, посещающих два курса - математику (`Math`) и поргутальский язык (`Por`). Некоторые студенты представлены в обоих курсах, некоторые - только в одном. Для каждого студента известны три оценки по курсу: оценка за первое полугодие (`G1`), оценка за второе полугодие (`G2`) и итоговая оценка за год (`G3`).

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 40)

pd.set_option('display.max_colwidth', None)

In [ ]:
data = pd.read_csv("students_data.csv")

data.shape

In [ ]:
data.head(15)

### Признаки

Данные представлены признаками различных типов: числовыми, категориальными, упорядоченными категориальными.

**Описание признаков:**

In [ ]:
pd.read_csv('students_data_features.csv',
            delimiter=';',
            encoding='windows-1251')

## Часть 1. Предобработка данных

* Разделите данные на две части - данные для моделирования (80%) и отложенную выборку (20%). Убедитесь, что распределение целевой переменной (`G3`) одинаково в обоих частях.  
  __NB__: Отложенную выборку нужно использовать только для финальной оценки качества модели. Обучение и кросс-валидацию следует проводить на данных для моделирования.  
* Выполните необходимые преобразования данных: исправление ошибок, удаление выбросов и пропусков, приведение признаков к числовому виду.  
* Оцените значимость признаков для определения итоговой оценки за курс. Исключите из выборки незначимые на ваш взгляд признаки, обоснуйте свое решение. 
* (Опционально) Feature engineering: создайте новые признаки (значимые) на основе уже имеющихся.
  
**Tip:** Используйте свои наработки из Лабораторной работы №1.

#### Перед разделением данных сделаем преобразование данных и исправим имеющиеся ошибки
Выведем список всех признаков и распределим по кортжеам в 3 категории

In [ ]:
print(data.dtypes)
categoric = ("Subject", "school", "sex", "address","famsize" ,"Pstatus", "Mjob", "Fjob", "reason", "guardian", "schoolsup", "famsup",
              "paid", "activities", "nursery", "higher", "internet", "romantic", "cheating")
numberic = ('ID', 'age', 'failures', 'absences', 'G1', 'G2', 'G3')
sort_categoric = ("Medu", "Fedu", "traveltime", "studytime", "famrel", "freetime", "goout", "Dalc", "Walc", "health")

Выведем уникальные значения для категориальных и упорядоченные категориальные. Найдем среди них "плохие" значения. Для числовых просто удостоверимся, что нет не числовых значений.

In [ ]:
print("Sort Categorical Values:")
for value in sort_categoric:
    print(value, data[value].unique())
print()
print("Categorical Values:")
for value in categoric:
    print(value, data[value].unique())

#### Теперь исправим имеющиеся ошибки

In [ ]:
data.loc[data['Medu'] == 'o', 'Medu'] = '0'
data.loc[data['Fedu'] == 'o', 'Fedu'] = '0'
data.loc[data['sex'] == 'm', 'sex'] = 'M'
data.loc[data['Pstatus'] == 't', 'Pstatus'] = 'T'
data.loc[data['Mjob'] == 'at-home', 'Mjob'] = 'at_home'
data.loc[data['Fjob'] == 'at-home', 'Fjob'] = 'at_home'
data.loc[data['guardian'] == 'futher', 'guardian'] = 'father'

print("Sort Categorical Values:")
for value in sort_categoric:
    print(value, data[value].unique())
print()
print("Categorical Values:")
for value in categoric:
    print(value, data[value].unique())

#### Также исправим типы некоторых значений

In [ ]:
import numpy as np

In [ ]:

data[['Medu', 'Fedu']] = data[['Medu', 'Fedu']].astype(np.int64)
print(data.dtypes)

#### Исправим проблему с пустыми ячейками, удалив столбцы с большим количеством пропущенных значений, а некоторые заменив на среднее значение

In [ ]:
print(data.isnull().sum())

In [ ]:
columns = data.columns.values.tolist()
empty_columns = list()
for value in columns:
    if data[value].isnull().sum() != 0:
        if(data[value].dtypes == 'object'):    
            data = data.fillna({value : data[value].value_counts().index[0]})
        else:
            data = data.fillna({value : data[value].median()})
data.drop(columns=["cheating"], inplace=True)
print(data.isnull().sum())
print(data.dtypes)

In [ ]:
data['Walc'] = data['Walc'].apply(int, convert_dtype=True)
data['Dalc'] = data['Dalc'].apply(int, convert_dtype=True)
data['famrel'] = data['famrel'].apply(int, convert_dtype=True)

### Выполним кодирование для категориальных переменных (Label Encoding)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
def countable(df=data):
    all_categories = list(categoric)
    all_categories.remove("cheating")
    all_categories_dict = dict()
    for value in all_categories:
        all_categories_dict[value] = set(data[value].unique())
    for i in all_categories_dict:
        all_categories_dict[i] = (all_categories_dict[i], len(all_categories_dict[i]))
    return all_categories_dict

data_enc_label = data[[column for column in data.columns if data[column].dtypes == 'object']].copy()
categ = [column for column in data.columns if data[column].dtypes == 'object']
labelencoder = LabelEncoder()
for i in categ:
    data_enc_label[i] = labelencoder.fit_transform(data_enc_label[i])
data_enc_label

### Теперь выполним One Hot Encoding

In [ ]:
data_enc_label.columns

In [ ]:
# One hot encoding
hot = OneHotEncoder(handle_unknown='ignore')
categories_data = countable(data[[column for column in data.columns if data[column].dtypes == 'object']].copy())
one_hot_df = pd.DataFrame(hot.fit_transform(data_enc_label[['Mjob', 'Fjob', 'guardian', 'reason']]).toarray())
one_hot_df.columns = [column + '_'+ str(list(categories_data[column][0])[i]) for column in ['Mjob', 'Fjob', 'guardian', 'reason']
                      for i in range(categories_data[column][1])]
for column in one_hot_df.columns:
     one_hot_df[column] = one_hot_df[column].apply(int, convert_dtype=True)
        
data_enc_label = data_enc_label.drop(['Mjob', 'Fjob', 'guardian', 'reason'], axis=1)
data_enc_label.columns = [column + '_'+ str(list(categories_data[column][0])[1]) for column in data_enc_label.columns]
data_object_encoded = data_enc_label.join(one_hot_df)
encoded_df = data_enc_label.join(one_hot_df)
encoded_df

In [ ]:
labels = data['G3']
objects = [label for label in data.columns if data[label].dtypes == 'object']
l_data = data.drop(columns=['ID','G3'])
l_data = l_data.drop(columns=objects)
l_data = l_data.join(data_object_encoded)
l_data

In [ ]:
l_data_out_G1 = l_data.drop(columns=['G1'])
l_data_out_G1.shape[1]

### Feature engineerign 
Рассмотрим коррреляцию признаков

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
plt.figure(figsize=(50, 50))
sns.heatmap(l_data.corr(), annot=True, fmt=".2f", linewidths=.5)

### Признаки с высокой корреляцией
Medu и Fedu
Walc и Dalc
При этом можно их можно выразить путем объединения
То есть общее отношение к алкоголю и образование родителей

In [ ]:
l_data['Сalc'] = l_data.loc[:, ['Walc', 'Dalc']].sum(1)
l_data['Pedu'] = l_data.loc[:, ['Fedu', 'Medu']].sum(1)
l_data = l_data.drop(columns=['Walc', 'Dalc', 'Fedu', 'Medu'])
l_data.head()

In [ ]:
l_data

In [ ]:
labels

In [ ]:
from sklearn.model_selection import train_test_split
y = labels

x_train, x_test, y_train, y_test = train_test_split(l_data, y, test_size=0.2)
#x_out_g1_train, x_out_g1_test, y_train, y_test = train_test_split(x_out_g1, y, test_size=0.2)


In [ ]:
x_train.head(10)

In [ ]:
y_train.describe()

In [ ]:
fig = sns.kdeplot(y_train, shade=True, color="b")
fig = sns.kdeplot(y_test, shade=True, color="r")
plt.show()

### Распределение G3 одинаково

Также сделаем стандартизацию данных

In [ ]:
# scal = StandardScaler()
# x_train = scal.fit_transform(x_train)
# x_test = scal.transform(x_test)
# x_out_g1_train = scal.fit_transform(x_out_g1_train)
# x_out_g1_test = scal.transform(x_out_g1_test)

## Часть 2. Регрессия

* Решите задачу регрессии: постройте модель, предсказывающую итоговую оценку, которую получит студент по предмету (`G3`). При решении задачи **нельзя** использовать признак `G2`.  
<br>  
* Для решения задачи примените следующие методы:  
  * Линейная регрессия + регуляризации  
  * Полиномиальная регрессия  
  * KNN  
  * Деревья решений, Random Forest  
  
  Для каждого метода выполните настройку гиперпараметров на кросс-валидации.  
<br>    
* Оцените качество каждой модели на отложенной выборке, используйте различные метрики. Сравните модели и сделайте вывод о качестве решения задачи.  
<br>    
* Задачу необходимо решить в двух вариантах: с использованием признака `G1`  и без него. Сравните качество решений в двух случаях.  
<br>    
* В регрессионных моделях попробуйте дать интерпретацию весам признаков. 

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV

In [ ]:
def print_metrics(current, prediction):
    print("MSE ", metrics.mean_squared_error(current, prediction))
    print("RMSE ", np.sqrt(metrics.mean_squared_error(current, prediction)))
    print("R2 ", metrics.r2_score(current, prediction))
    print("MAE ", metrics.mean_absolute_error(current, prediction))

In [ ]:
x_train

In [ ]:
x_train, x_test, y_train, y_test
x_train_out_G2 = x_train.drop("G2", axis=1)
x_test_out_G2 = x_test.drop("G2", axis=1)

#### With G1

In [ ]:
from sklearn import metrics

### Linear Regression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(x_train_out_G2, y_train)
y_pred = lin_reg.predict(x_test_out_G2)
print("Linear Regression:")
print_metrics(y_test, y_pred)

### Ridge Regression

In [ ]:
lambdas = [0, 0.1, 0.5, 1, 5, 10, 25, 50, 100, 250, 500, 1000, 5000, 10000]
ridgecv = RidgeCV(alphas=lambdas, cv=5)
ridgecv.fit(x_train_out_G2, y_train)
y_pred = ridgecv.predict(x_test_out_G2)
print("Ridge regression")
print_metrics(y_test, y_pred)
print("The best Ridge: ", ridgecv.alpha_)

In [ ]:
print(len(ridgecv.coef_))
print(len(x_train_out_G2.columns))

Судя по весам предсказание делается из studytime, G1 

### Lasso Regression

In [ ]:
lasso = LassoCV(alphas=np.arange(0.1, 10, 0.1), normalize=True, cv=5)
lasso.fit(x_train_out_G2, y_train)
y_pred = lasso.predict(x_test_out_G2)
print("Lasso regression")
print_metrics(y_test, y_pred)
print(f"The best Lasso : ", lasso.alpha_)

#### Without C1

In [ ]:
x_train_out_G2_G1 = x_train_out_G2.drop('G1', axis=1)
x_test_out_G2_G1 = x_test_out_G2.drop('G1', axis=1)

### Linear Regression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(x_train_out_G2_G1, y_train)
y_pred = lin_reg.predict(x_test_out_G2_G1)
print("Linear Regression:")
print_metrics(y_test, y_pred)

### Lasso Regression

In [ ]:
lasso = LassoCV(alphas=np.arange(0.1, 10, 0.1), normalize=True, cv=5)
lasso.fit(x_train_out_G2_G1, y_train)
y_pred = lasso.predict(x_test_out_G2_G1)
print("Lasso regression")
print_metrics(y_test, y_pred)
print(f"The best Lasso : ", lasso.alpha_)

In [ ]:
print(lasso.coef_)
print(len(x_test_out_G2_G1.columns))
#print_weights(lasso, x_test_out_G2_G1.columns)

### Ridge Regression

In [ ]:
lambdas = [0, 0.1, 0.5, 1, 5, 10, 25, 50, 100, 250, 500, 1000, 5000, 10000]
ridgecv = RidgeCV(alphas=lambdas, cv=5)
ridgecv.fit(x_train_out_G2_G1, y_train)
y_pred = ridgecv.predict(x_test_out_G2_G1)
print("Ridge regression")
print_metrics(y_test, y_pred)
print("The best Ridge: ", ridgecv.alpha_)

## Polynomial Regression

### With G1

In [ ]:
degrees = [2, 3, 4]
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

In [ ]:
for i in degrees:
    pol = PolynomialFeatures(degree=i, include_bias=False)
    pol.fit(x_train_out_G2)
    pol_train = pol.transform(x_train_out_G2)
    pol_test = pol.transform(x_test_out_G2)
    pol_reg_model = linear_model.LinearRegression(normalize=True)
    pol_reg_model.fit(pol_train, y_train)
    y_pred = pol_reg_model.predict(pol_test)
    print("Degree ", i)
    print_metrics(y_test, y_pred)
    print()

## Without G1

In [ ]:
for i in degrees:
    pol = PolynomialFeatures(degree=i, include_bias=False)
    pol.fit(x_train_out_G2_G1)
    pol_train = pol.transform(x_train_out_G2_G1)
    pol_test = pol.transform(x_test_out_G2_G1)
    pol_reg_model = linear_model.LinearRegression(normalize=True)
    pol_reg_model.fit(pol_train, y_train)
    y_pred = pol_reg_model.predict(pol_test)
    print("Degree ", i)
    print_metrics(y_test, y_pred)
    print()

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

#### With G1

In [ ]:
parametrs = {'n_neighbors': range(1, 70), 
             'weights':['distance', 'uniform']}
grid = GridSearchCV()

## Часть 3. Бинарная классификация

Решите задачу бинарной классификации: постройте модель, предсказывающую, сдаст студент предмет (`G3` >= 8) или не сдаст (`G3` < 8). <br>При решении задачи **нельзя** использовать признаки `G1` и `G2`.  


### Задание 1  
  
* Постройте дерево решений глубины 5 (остальные параметры по умолчанию), оцените качество на 5-fold валидации.  
* Для одного из деревьев (т.е. обученного на одной из итераций кросс-валидации) выведите само дерево - постройте график или выведите в текстовом виде. По структуре дерева сформулируйте правила, по которым принимается решение.  
* Сравните между собой деревья решений, полученных на различных итерациях 5-fold валидации. Сделайте вывод, насколько сильно они похожи или различаются между собой. 

In [ ]:
# your code here

### Задание 2  
  
На кросс-валидации (5-fold из 2 повторений) оцените, как меняется качество модели Random Forest с ростом числа деревьев (при дефолтных значениях остальных параметров). Провизуализируйте результаты. Сколько деревьев достаточно в данном случае и почему?  
**NB:** В сравнение включите конфигурацию, аналогичную простому дереву решений. 

In [ ]:
# your code here

### Задание 3  
  
* Настройте гиперпараметры модели Random Forest на 5-fold валдиации. В качестве метрики используйте F1-score. Замерьте время, затраченное на вычисления.
* Обучите Random Forest  с настроенными параметрами на всех данных для моделирования. На отложенной выборке оцените качество (F1-score) всего ансамбля и <u>каждого дерева отдельно</u>. Постройте график распределения качества деревьев в ансамбле и сравните результаты с качеством всего леса. Дайте комментарий.  
* Выведите важность признаков в Random Forest, сделайте выводы. 

In [ ]:
# your code here

### Задание 4  
  
* Примените логистическую регрессию для решения задачи, подберите оптимальные значения гиперпараметров. Оцените качество (roc auc) на 5-fold валидации из 2 повторений. 
* Аналогично (на такой же валидации (тех же подвыборках) с такой же метрикой) оцените качество Random Forest  с подобранными в предыдущем задании параметрами. Сравните с качеством логистическом регрессии.
* Обучите логистическую модель с настроенными параметрами на всех данных для моделирования. На отложенной выборке оцените качество - постройте ROC-кривую, вычислите roc auc. Вычислите аналогичную метрику для Random Forest из Задания 3, сравните точность моделей. 

In [ ]:
# your code here

### Задание 5  
  
* Используйте для решения задачи один из фреймворков градиентного бустинга: XGBoost, LightGDB или CatBoost.  
* Оцените на 5-fold валидации, как растет качество модели на обучающей и на тестовой выборках при добавлении каждого дерева. Провизуализируйте результаты.  
* Настройте гиперпараметры модели на 5-fold валидации, в качестве метрики используйте F1-score. Замерьте время, затраченное на вычисления.  
* Обучите модель с настроенными параметрами на всех данных для моделирования и оцените качество на отложенной выборке. Сравните результаты с другими моделями, дайте комментарий.

In [ ]:
# your code here

##  Часть 4. Многоклассовая классификация
  
* Решите задачу многоклассовой классификации: постройте модель, пресдказывающую оценку студента по предмету по 4 балльной шкале
    - Отлично: 18 <= `G3` <= 20
    - Хорошо: 14 <= `G3` <= 17
    - Удовлетворительно: 8 <= `G3` <= 13
    - Неудовлетворительно: `G3` < 8  
  
  При решении задачи **нельзя** использовать признаки `G1` и `G2`.  
  
  
* Для решения задачи примените следующие методы:  
  * KNN  
  * Логистическая регрессия  
  * Деревья решений  
  * Random Forest
  * Gradient Boosting
  
  На кросс-валидации подберите оптимальные значения гиперпараметров алгоритмов.  
  
  
* Оцените качество моделей, используйте confusion matrix и производные от нее метрики. Сделайте выводы.    

In [ ]:
# your code here